In [1]:
# pip install scikit-learn
# pip install keras
# pip install tensorflow


import pandas as pd
import numpy as np
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import Sequence
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import load_model


# pd.set_option('display.max_columns', 100)



: 

: 

In [ ]:
df = pd.read_csv("prueba.csv")

df.drop(columns=['id','date_time_utc','unix_time'], inplace=True)


df.head(10)

In [ ]:

def data_preparation(df):

    # drop the datetime column
    df_reduced = df.drop(columns=['datetime'])

    scaler = MinMaxScaler()

    # Fit the scaler on your dataframe (let's say it's called df)
    df_normalized = scaler.fit_transform(df_reduced)

    # Convert the normalized data back to a dataframe
    df_normalized = pd.DataFrame(df_normalized, columns=df_reduced.columns)

    # Convert the DataFrame to a NumPy array
    raw_data = df_normalized.values
    close = df_normalized['close_price'].values
    close = close.reshape((len(close),1))

    # Display the NumPy array
    print(type(raw_data))
    print(raw_data.shape)
    print(close.shape)

    return raw_data, close



def predict(df,model_path,delay,sampling_rate,sequence_length):

    raw_data, close = data_preparation(df)

    # Set Parameters

    # sequence_stride: period between sequences
    # First sequence starts at t0
    # Second sequence will start at t1 with sequence_stride=1 or at t5 with sequence_stride=5
    sequence_stride = 1

    #batch_size: Number of timeseries samples in each batch (except maybe the last one). 
    #If None, the data will not be batched (the dataset will yield individual samples).
    # Huge impact in performance.
    # Tip, should be multiple of 8
    batch_size = 32

    # Understanding our parameters
    msg = f"The timeseries will consist of batches containing {batch_size} sequences of {sequence_length} samples."

    msg += f"\nFinally our target is {delay} timesteps in the future, and will have data from {sequence_length * sampling_rate} timesteps in the past"
    print(msg)

    keras_dataset = keras.preprocessing.timeseries_dataset_from_array(
                        raw_data[:-delay],
                        targets=close[delay:],
                        sampling_rate=sampling_rate,
                        sequence_stride=sequence_stride,
                        sequence_length=sequence_length,
                        shuffle=False, # Shouldn't the shuffle be set to 0?
                        seed=33,
                        batch_size=batch_size,
                        start_index=0)
    
    # Load the model from the .h5 file
    modelo =  load_model(model_path)

    test_pred = modelo.predict(keras_dataset)

    # Assume 'y_normalized' contains the predicted values for the 'target' column in normalized form
    y_min = df['close_price'].min()
    y_max = df['close_price'].max()

    test_pred = test_pred * (y_max - y_min) + y_min

    predicted_prices = pd.DataFrame(test_pred, columns=['close_price'])

    return predicted_prices

